In [17]:
!pip install imageio-ffmpeg openai requests



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import time
from pathlib import Path
import requests
from openai import OpenAI
import imageio.v3 as imageio
import os

In [2]:
# List of all countries recognized by the United Nations in alphabetical order
# Note: Sudan and South Sudan have been excluded because of OpenAI's policy.

un_countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia", "Australia", 
    "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", 
    "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Burundi", 
    "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China", "Colombia", 
    "Comoros", "Congo, Democratic Republic of the", "Congo, Republic of the", "Costa Rica", "Croatia", "Cuba", "Cyprus", 
    "Czech Republic", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador", 
    "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", 
    "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", 
    "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Jamaica", 
    "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Korea, North", "Korea, South", "Kosovo", "Kuwait", "Kyrgyzstan", 
    "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", 
    "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", 
    "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", 
    "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Macedonia", "Norway", "Oman", 
    "Pakistan", "Palau", "Palestine", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", 
    "Portugal", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", 
    "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", 
    "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "Spain", "Sri Lanka", "Suriname", 
    "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste", 
    "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", 
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City", 
    "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

In [ ]:
# Initialize OpenAI client
client = OpenAI()

# Create a 'people' folder in the same directory where the script is run
folder_path = Path("./people")
folder_path.mkdir(parents=True, exist_ok=True)

# Function to check if an image for a country already exists
def image_exists(country):
    return (folder_path / f"{country}.png").exists()

# Function to generate an image using OpenAI's DALL-E and save it
def generate_image_for_country(country):
    prompt = f"A picture of a person from {country}, with the country's name on it."

    # Call to OpenAI's DALL-E API
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="hd",
        n=1,
    )
    
    # Get the URL of the generated image
    image_url = response.data[0].url

    # Download the image from the URL
    response = requests.get(image_url)
    if response.status_code == 200:
        # Save the image
        image_path = folder_path / f"{country}.png"
        with open(image_path, "wb") as file:
            file.write(response.content)
        return str(image_path)
    else:
        print(f"Failed to download image for {country}")
        return None

# Function to generate and save images with rate limiting
def generate_and_save_images(countries, rate_limit=4):
    generated_paths = []
    for i, country in enumerate(countries):
        if image_exists(country):
            print(f"Image for {country} already exists. Skipping.")
            continue

        # Generate and save image
        image_path = generate_image_for_country(country)
        if image_path:
            generated_paths.append(image_path)

        # Rate limiting: Sleep after every 4th image, but not after the last image
        if (i + 1) % rate_limit == 0 and (i + 1) != len(countries):
            print(f"Rate limit reached. Sleeping for 60 seconds.")
            time.sleep(60)

    return generated_paths

# Generate and save images for all UN countries
generated_images = generate_and_save_images(un_countries)


In [18]:
# Directory containing the images
folder_path = "./people"

# List of image filenames (make sure they are sorted in the order you want them to appear in the video)
image_filenames = sorted([file for file in os.listdir(folder_path) if file.endswith(".png")])

# Load the images
images = [imageio.imread(os.path.join(folder_path, filename)) for filename in image_filenames]

# Create the output video file
imageio.imwrite("countries_video.mp4", images, fps=2)  # Adjust FPS as needed
